In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import cv2
from skimage.transform import resize
from skimage.io import imread
from sklearn.utils import shuffle
import random
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

2023-03-14 16:18:29.977466: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preprocessing Image Data 

In [3]:
Categories=['rock','paper','scissors']
train_dir = '/Users/hanyijia/Desktop/Midterm_Project/rps-split/train'
test_dir = '/Users/hanyijia/Desktop/Midterm_Project/rps-split/test'
val_dir = '/Users/hanyijia/Desktop/Midterm_Project/rps-split/val'

flat_data_arr=[] #input array
target_arr=[] #output array
for i in Categories:
    path=os.path.join(train_dir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(150,150,1))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully for train')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x_train=df.iloc[:,:-1] #input data 
y_train=df.iloc[:,-1] #output data
x_train=x_train.sample(frac=1, random_state=42).reset_index(drop=True)
y_train=y_train.sample(frac=1, random_state=42).reset_index(drop=True)



flat_data_arr=[] #input array
target_arr=[] #output array
for i in Categories:
    path=os.path.join(test_dir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(150,150,1))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully for test')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x_test=df.iloc[:,:-1] #input data 
y_test=df.iloc[:,-1] #output data

flat_data_arr=[] #input array
target_arr=[] #output array
for i in Categories:
    path=os.path.join(val_dir,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(150,150,1))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{i} successfully for val')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x_val=df.iloc[:,:-1] #input data 
y_val=df.iloc[:,-1] #output data

loaded category:rock successfully for train
loaded category:paper successfully for train
loaded category:scissors successfully for train
loaded category:rock successfully for test
loaded category:paper successfully for test
loaded category:scissors successfully for test
loaded category:rock successfully for val
loaded category:paper successfully for val
loaded category:scissors successfully for val


In [5]:
df

,0,1,2,3,4,5,6,7,8,9,...,22491,22492,22493,22494,22495,22496,22497,22498,22499,Target
0,0.389626,0.393559,0.391569,0.395579,0.395402,0.397788,0.398471,0.399835,0.400849,0.403261,...,0.422659,0.423343,0.415783,0.409807,0.409548,0.410292,0.413908,0.414923,0.411938,0
1,0.303303,0.299762,0.301882,0.299680,0.303934,0.304589,0.311469,0.308152,0.308210,0.314294,...,0.331119,0.330005,0.331700,0.321468,0.324136,0.326494,0.323933,0.318195,0.321130,0
2,0.295309,0.299908,0.304238,0.308415,0.307158,0.310090,0.317037,0.316009,0.324760,0.324716,...,0.214789,0.217025,0.214020,0.209220,0.204367,0.200442,0.196713,0.196020,0.186629,0
3,0.407398,0.409533,0.402261,0.407455,0.407025,0.410977,0.412065,0.410720,0.411594,0.414866,...,0.369783,0.368859,0.361380,0.358445,0.357558,0.356558,0.358704,0.361119,0.361117,0
4,0.280727,0.280187,0.276002,0.276598,0.278571,0.283833,0.285883,0.284404,0.279927,0.288574,...,0.301772,0.304143,0.297735,0.295623,0.297901,0.299526,0.300013,0.300577,0.297344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,0.252707,0.259578,0.265256,0.276400,0.280335,0.278996,0.288753,0.294613,0.301661,0.302332,...,0.252940,0.249484,0.243592,0.238724,0.231929,0.228928,0.224893,0.221234,0.211138,2
324,0.271310,0.277713,0.281664,0.287216,0.290723,0.293748,0.302709,0.305199,0.311924,0.314540,...,0.295237,0.291964,0.288177,0.286145,0.283898,0.276415,0.275052,0.272200,0.259858,2
325,0.274138,0.275507,0.276563,0.272706,0.278246,0.275015,0.280245,0.279506,0.281945,0.282377,...,0.328828,0.330404,0.330226,0.325948,0.323331,0.330867,0.327988,0.325868,0.325927,2
326,0.259139,0.266640,0.268944,0.278137,0.283420,0.284890,0.293491,0.298141,0.305742,0.309923,...,0.262509,0.259193,0.255458,0.250740,0.242482,0.241378,0.236497,0.231195,0.220456,2


In [12]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,22490,22491,22492,22493,22494,22495,22496,22497,22498,22499
0,0.236356,0.233876,0.235644,0.240776,0.243214,0.250443,0.267286,0.256402,0.255382,0.258343,...,0.327829,0.323856,0.321208,0.324128,0.312872,0.315432,0.315056,0.308026,0.304768,0.306284
1,0.303605,0.295018,0.303132,0.301293,0.303537,0.305217,0.307818,0.305904,0.307678,0.308938,...,0.335613,0.338397,0.333728,0.330845,0.327453,0.326771,0.328089,0.319492,0.317175,0.320230
2,0.315842,0.309967,0.310030,0.311028,0.314933,0.318514,0.327569,0.321398,0.318474,0.323595,...,0.355158,0.352948,0.351521,0.348997,0.339114,0.341553,0.347256,0.342291,0.336760,0.342175
3,0.423149,0.422035,0.424096,0.431453,0.437395,0.439167,0.435639,0.439682,0.431962,0.440786,...,0.382692,0.385339,0.381970,0.376206,0.365648,0.369896,0.367752,0.369671,0.366388,0.362932
4,0.323088,0.310471,0.315167,0.315875,0.316624,0.321134,0.324729,0.319467,0.318575,0.327514,...,0.345591,0.344400,0.337880,0.336033,0.335818,0.336489,0.334532,0.329182,0.325635,0.325154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526,0.309414,0.314598,0.311858,0.315044,0.316247,0.324053,0.320759,0.324037,0.320585,0.327300,...,0.340113,0.343950,0.345809,0.338292,0.331292,0.334437,0.332917,0.334826,0.335207,0.331315
1527,0.310359,0.306660,0.311521,0.316477,0.315204,0.317298,0.323702,0.321015,0.324634,0.323515,...,0.329871,0.332405,0.327741,0.328140,0.321077,0.323985,0.327053,0.319829,0.319067,0.319303
1528,0.356962,0.356470,0.359184,0.357502,0.364072,0.367306,0.366079,0.362692,0.367178,0.369281,...,0.366552,0.369752,0.364943,0.364344,0.365332,0.364064,0.363169,0.360957,0.357530,0.357354
1529,0.418303,0.421090,0.417992,0.423174,0.424857,0.426658,0.430374,0.429240,0.422659,0.431986,...,0.377305,0.380088,0.381242,0.371545,0.363513,0.366670,0.367815,0.372163,0.371923,0.377429


In [42]:
y_train

0      0
1      0
2      0
3      0
4      0
      ..
324    2
325    2
326    2
327    2
328    2
Name: Target, Length: 329, dtype: int64

# Model Construction

In [31]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis()


In [32]:
X_lda = model.fit(x_train, y_train).transform(x_train)
y_pred= model.predict(x_train)
model.decision_function(x_train)



array([[-11.81582595,   7.58616354,  -8.96185306],
       [  2.82733358,  -5.78455613, -10.44777048],
       [  1.09580022,  -8.07339806,  -6.60117576],
       ...,
       [ -7.10472841,   2.35372255,  -8.55705675],
       [-13.83584891,  -4.25375618,   4.22376919],
       [ -8.97049026,  -2.05862887,  -2.56627956]])

In [58]:
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_train, y_pred)
print(cm1)

[[474  13  21]
 [  3 484  11]
 [  9   7 509]]


In [59]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95       508
           1       0.96      0.97      0.97       498
           2       0.94      0.97      0.95       525

    accuracy                           0.96      1531
   macro avg       0.96      0.96      0.96      1531
weighted avg       0.96      0.96      0.96      1531



In [60]:
y_pred2 = model.predict(x_test)
model.decision_function(x_test)


array([[ 3.90301525e+00, -1.09530647e+01, -6.58592099e+00],
       [-3.35621301e+00, -3.28337023e+00, -6.83700651e+00],
       [-3.17830037e+00, -6.02203515e+00, -4.41133887e+00],
       [ 7.08826282e+00, -9.76610438e+00, -1.07939439e+01],
       [ 7.12791251e+00, -2.00712328e+01, -1.05715929e+00],
       [-1.26840749e+00, -6.61133034e+00, -5.70039901e+00],
       [-3.87712891e+00, -1.51345267e+01,  4.90871004e+00],
       [-4.15085319e+00, -2.67858124e+00, -6.64178308e+00],
       [ 2.93180721e+00, -1.05703167e+01, -6.00922545e+00],
       [-5.73329673e+00, -3.49015521e+00, -4.34074470e+00],
       [ 7.32527401e+00, -1.21426181e+01, -8.76898736e+00],
       [-2.02891663e+00, -4.40728532e+00, -7.05521002e+00],
       [ 9.73708700e+00, -1.10516964e+01, -1.21375208e+01],
       [ 8.81033639e-01, -9.85144483e+00, -4.70675916e+00],
       [-2.41876933e+00, -7.23096378e+00, -3.99952041e+00],
       [ 6.33346902e+00, -1.48633735e+01, -5.22846714e+00],
       [ 4.71192113e+00, -1.03386493e+01

In [63]:
cm2 = confusion_matrix(y_test, y_pred2)

print(cm2)
print(classification_report(y_test, y_pred2))

[[85  8 16]
 [19 67 21]
 [23 18 72]]
              precision    recall  f1-score   support

           0       0.67      0.78      0.72       109
           1       0.72      0.63      0.67       107
           2       0.66      0.64      0.65       113

    accuracy                           0.68       329
   macro avg       0.68      0.68      0.68       329
weighted avg       0.68      0.68      0.68       329

